In [3]:
import pandas as pd
import numpy as np


In [4]:
db_path = r'C:\GitHub\DATABASE_ORIG\new_versionDB_indexcheck.xlsx'

# db_path = r"C:\Users\xbacos\OneDrive - University of Gothenburg\here.xlsx"

In [ ]:
db_sh = pd.ExcelFile(db_path)
sheets = db_sh.sheet_names
db = pd.read_excel(db_path, sheet_name= sheets, index_col= 0)
# add 

a = 1
for col in sheets:
    if col == 'Name':
        db[col]['nameOrigin'] = db[col]['Name'].astype(str) + ', ' + db[col]['Origin'].astype(str)
    elif col == 'References': 
        db[col]['authorYear'] = db[col]['Author'].astype(str) + ', ' + db[col]['Year'].astype(str)
    elif col == 'Country':
        db[col]['nameOrigin'] = db[col]['Country'].astype(str) + ', ' + db[col]['City'].astype(str)  
    elif col == 'Region':
        pass
    elif col == 'Spartacus Material':
        db[col]['nameOrigin'] = db[col]['Name'].astype(str) + '; ' + db[col]['Color'].astype(str) + '; ' + db[col]['Origin'].astype(str)    
    # Calculate U-values for roof and wall new columns u_value_wall and u_value_roof
    elif col == 'Spartacus Surface':
        db[col]['nameOrigin'] = db[col]['Name'].astype(str) + ', ' + db[col]['Origin'].astype(str)
            # Filter rows where Surface is 'Buildings'
       
        buildings = db['Spartacus Surface'][db['Spartacus Surface']['Surface'] == 'Buildings']

        # Calculate resistances and U-values
        for prefix in ['w', 'r']:

            if prefix == 'w':
                pr = 'wall'
            else:
                pr = 'roof'
            materials = buildings[[f'{prefix}{i}Material' for i in range(1, 6)]].values
            thicknesses = buildings[[f'{prefix}{i}Thickness' for i in range(1, 6)]].values

            thicknesses[np.isnan(thicknesses)] = 0

            for i in range(0,5):
                materials[np.isnan(materials)] = materials[np.nonzero(np.isnan(materials))[0], np.nonzero(np.isnan(materials))[1]-1]


            thermal_conductivities = np.vectorize(lambda x: db['Spartacus Material'].loc[x, 'Thermal Conductivity'])(materials)


            resistances = thicknesses / thermal_conductivities
            resistance_bulk = resistances.sum(axis=1)

            u_values = 1 / resistance_bulk

            db['Spartacus Surface'].loc[buildings.index, f'u_value_{pr}'] = u_values

        # Calculate albedo and emissivity
        for prop in ['Albedo', 'Emissivity']:
            for prefix in ['w', 'r']:
                if prefix == 'w':
                    pr = 'wall'
                elif prefix == 'r':
                    pr == 'roof'
                material_col = f'{prefix}1Material'
                db['Spartacus Surface'].loc[buildings.index, f'{prop.lower()}_{pr}'] = db['Spartacus Material'].loc[buildings[material_col], prop].values
    
    elif col == 'Profiles':
        # Normalise traffic and energy use profiles to ensure that average of all columns = 1
        normalisation_rows = db[col][(db[col]['Profile Type'] == 'Traffic') | (db[col]['Profile Type'] == 'Energy use')]
        cols = list(range(24))
        normalisation_rows_index = list(normalisation_rows.index)

        # # # Calculate the sum of the values for each row
        sums = db[col].loc[normalisation_rows_index, cols].sum(axis=1)

        # Avoid division by zero by replacing zero sums with NaN
        sums.replace(0, np.nan, inplace=True)

        # # Calculate the scaling factor to make the sum equal to the number of columns (24)
        scaling_factors = 24 / sums

        # Scale the values
        db[col].loc[normalisation_rows_index, cols] = db[col].loc[normalisation_rows_index, cols].multiply(scaling_factors, axis=0)
        
        # Create unique name
        db[col]['nameOrigin'] = db[col]['Name'].astype(str)  +  ', ' + db[col]['Day'].astype(str) +  ', ' + db[col]['Country'].astype(str) + ', ' + db[col]['City'].astype(str) 

    else:
        # Standard
        db[col]['nameOrigin'] = db[col]['Name'].astype(str) + ', ' + db[col]['Origin'].astype(str)
    
db_sh.close() # trying this to close excelfile


def save_to_db(db_path, db_dict):
    # Drop columns in a 
    for col in db_dict.keys():
        if col == 'References':
            db_dict[col] = db_dict[col].drop(columns='authorYear', errors='ignore')
        elif col not in ['Country', 'Region']:
            db_dict[col] = db_dict[col].drop(columns='nameOrigin', errors='ignore')

    # Save to Excel
    with pd.ExcelWriter(db_path) as writer:
        for sheet_name, df in db_dict.items():
            df.to_excel(writer, sheet_name=sheet_name)

    # Add 'nameOrigin' and 'authorYear' columns back
    if 'References' in db_dict:
        db_dict['References']['authorYear'] = db_dict['References']['Author'].astype(str) + ', ' + db_dict['References']['Year'].astype(str)
    if 'Profiles' in db_dict:
        db_dict['Profiles']['nameOrigin'] = db_dict['Profiles']['Name'].astype(str)  +  ', ' + db_dict['Profiles']['Day'].astype(str) +  ', ' + db_dict['Profiles']['Country'].astype(str) + ', ' + db_dict['Profiles']['City'].astype(str) 
    for col in db_dict.keys():
        if col not in ['Profiles', 'References', 'Country', 'Region']:
            db_dict[col]['nameOrigin'] = db_dict[col]['Name'].astype(str) + ', ' + db_dict[col]['Origin'].astype(str)


u_value_wall
u_value_roof


In [17]:
db['Spartacus Surface']

,Origin,Surface,Name,r1Material,r1Thickness,r2Material,r2Thickness,r3Material,r3Thickness,r4Material,...,w4Material,w4Thickness,w5Material,w5Thickness,Ref,nameOrigin,u_value_wall,u_value_roof,albedo_wall,emissivity_wall
ID,,,,,,,,,,,,,,,,,,,,,
3729073437,Sweden,Buildings,"Traditional Neighbourhood, early 1900s",3625478178,0.0540,3625478155,0.0250,3.625478e+09,0.0250,3.625478e+09,...,3.625478e+09,0.0500,3.625478e+09,0.0500,9999,"Traditional Neighbourhood, early 1900s, Sweden",0.298716,0.433437,0.200,0.90
3729089758,Sweden,Buildings,"Nordic Functionalism, mid 1900s",3625478180,0.0540,3625478155,0.0250,3.625478e+09,0.0250,3.625478e+09,...,3.625478e+09,0.0500,3.625478e+09,0.0500,9999,"Nordic Functionalism, mid 1900s, Sweden",0.285956,0.447914,0.250,0.90
3729090260,Sweden,Buildings,"Million Program, mid 1900s",3625478156,0.0100,3625478145,0.0500,3.625478e+09,0.0700,3.625478e+09,...,3.625478e+09,0.0100,3.625478e+09,0.0100,9999,"Million Program, mid 1900s, Sweden",0.297697,0.330465,0.180,0.92
3729005194,Sweden,Buildings,"Mixed-City, early 200s",3625478156,0.2200,3625478168,0.0400,3.625478e+09,0.2600,3.625478e+09,...,3.625478e+09,0.2200,3.625478e+09,0.2200,9999,"Mixed-City, early 200s, Sweden",0.079402,0.192080,0.180,0.92
3730430893,Sweden,Buildings,"Detached houses, Wood",3625478178,0.0540,3625478155,0.0250,3.625478e+09,0.0150,3.625478e+09,...,3.625478e+09,0.0050,3.625478e+09,0.0050,9999,"Detached houses, Wood, Sweden",1.097377,0.804598,0.200,0.90
3730430894,UK,Buildings,Central London [sample run],3630433784,0.0540,3630433785,0.1500,3.630434e+09,0.0400,3.630434e+09,...,3.630434e+09,0.0497,3.630434e+09,0.0497,9057,"Central London [sample run], UK",0.464196,0.314830,0.500,0.95
3730430895,UK,Paved,Paved [sample run],3630433788,0.0400,3630433789,0.3000,3.630434e+09,1.2000,3.630434e+09,...,-9.990000e+02,-999.0000,-9.990000e+02,-999.0000,9057,"Paved [sample run], UK",NaN,NaN,NaN,NaN
3730430892,Sweden,Buildings,Industry,3630433783,0.0100,3625478145,0.0500,3.625478e+09,0.0300,3.625478e+09,...,3.630434e+09,0.0400,3.630434e+09,0.0400,9999,"Industry, Sweden",0.327795,0.659896,0.060,0.95
3730430896,UK,Buildings,"Residential, pre-1900-1944, pitched roof, type...",3625478178,0.0120,3625478158,0.0200,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"9069, 9070","Residential, pre-1900-1944, pitched roof, type...",3.241267,7.179487,0.200,0.90


In [15]:

db = pd.read_excel(db_path, sheet_name= sheets, index_col= 0)

buildings = db['Spartacus Surface'][db['Spartacus Surface']['Surface'] == 'Buildings']

# Calculate resistances and U-values
for prefix in ['w', 'r']:

    if prefix == 'w':
        pr = 'wall'
    else:
        pr = 'roof'
    materials = buildings[[f'{prefix}{i}Material' for i in range(1, 6)]].values
    thicknesses = buildings[[f'{prefix}{i}Thickness' for i in range(1, 6)]].values

    thicknesses[np.isnan(thicknesses)] = 0

    for i in range(0,5):
        materials[np.isnan(materials)] = materials[np.nonzero(np.isnan(materials))[0], np.nonzero(np.isnan(materials))[1]-1]


    thermal_conductivities = np.vectorize(lambda x: db['Spartacus Material'].loc[x, 'Thermal Conductivity'])(materials)


    resistances = thicknesses / thermal_conductivities
    resistance_bulk = resistances.sum(axis=1)

    u_values = 1 / resistance_bulk

    print(f'u_value_{pr}')
    db['Spartacus Surface'].loc[buildings.index, f'u_value_{pr}'] = u_values


db['Spartacus Surface']

u_value_wall
u_value_roof


,Origin,Surface,Name,r1Material,r1Thickness,r2Material,r2Thickness,r3Material,r3Thickness,r4Material,...,w2Thickness,w3Material,w3Thickness,w4Material,w4Thickness,w5Material,w5Thickness,Ref,u_value_wall,u_value_roof
ID,,,,,,,,,,,,,,,,,,,,,
3729073437,Sweden,Buildings,"Traditional Neighbourhood, early 1900s",3625478178,0.0540,3625478155,0.0250,3.625478e+09,0.0250,3.625478e+09,...,0.0254,3.625478e+09,0.0254,3.625478e+09,0.0500,3.625478e+09,0.0500,9999,0.298716,0.433437
3729089758,Sweden,Buildings,"Nordic Functionalism, mid 1900s",3625478180,0.0540,3625478155,0.0250,3.625478e+09,0.0250,3.625478e+09,...,0.0500,3.625478e+09,0.0200,3.625478e+09,0.0500,3.625478e+09,0.0500,9999,0.285956,0.447914
3729090260,Sweden,Buildings,"Million Program, mid 1900s",3625478156,0.0100,3625478145,0.0500,3.625478e+09,0.0700,3.625478e+09,...,0.1000,3.625478e+09,0.1200,3.625478e+09,0.0100,3.625478e+09,0.0100,9999,0.297697,0.330465
3729005194,Sweden,Buildings,"Mixed-City, early 200s",3625478156,0.2200,3625478168,0.0400,3.625478e+09,0.2600,3.625478e+09,...,0.1500,3.625478e+09,0.1300,3.625478e+09,0.2200,3.625478e+09,0.2200,9999,0.079402,0.192080
3730430893,Sweden,Buildings,"Detached houses, Wood",3625478178,0.0540,3625478155,0.0250,3.625478e+09,0.0150,3.625478e+09,...,0.0254,3.625478e+09,0.0150,3.625478e+09,0.0050,3.625478e+09,0.0050,9999,1.097377,0.804598
3730430894,UK,Buildings,Central London [sample run],3630433784,0.0540,3630433785,0.1500,3.630434e+09,0.0400,3.630434e+09,...,0.0497,3.630434e+09,0.0497,3.630434e+09,0.0497,3.630434e+09,0.0497,9057,0.464196,0.314830
3730430895,UK,Paved,Paved [sample run],3630433788,0.0400,3630433789,0.3000,3.630434e+09,1.2000,3.630434e+09,...,-999.0000,-9.990000e+02,-999.0000,-9.990000e+02,-999.0000,-9.990000e+02,-999.0000,9057,NaN,NaN
3730430892,Sweden,Buildings,Industry,3630433783,0.0100,3625478145,0.0500,3.625478e+09,0.0300,3.625478e+09,...,0.1000,3.630434e+09,0.0400,3.630434e+09,0.0400,3.630434e+09,0.0400,9999,0.327795,0.659896
3730430896,UK,Buildings,"Residential, pre-1900-1944, pitched roof, type...",3625478178,0.0120,3625478158,0.0200,NaN,NaN,NaN,...,0.0130,NaN,NaN,NaN,NaN,NaN,NaN,"9069, 9070",3.241267,7.179487


In [6]:
db['Spartacus Surface']

,Origin,Surface,Name,r1Material,r1Thickness,r2Material,r2Thickness,r3Material,r3Thickness,r4Material,...,w4Material,w4Thickness,w5Material,w5Thickness,Ref,nameOrigin,u_value_wall,u_value_rall,albedo_wall,emissivity_wall
ID,,,,,,,,,,,,,,,,,,,,,
3729073437,Sweden,Buildings,"Traditional Neighbourhood, early 1900s",3625478178,0.0540,3625478155,0.0250,3.625478e+09,0.0250,3.625478e+09,...,3.625478e+09,0.0500,3.625478e+09,0.0500,9999,"Traditional Neighbourhood, early 1900s, Sweden",0.298716,0.433437,0.200,0.90
3729089758,Sweden,Buildings,"Nordic Functionalism, mid 1900s",3625478180,0.0540,3625478155,0.0250,3.625478e+09,0.0250,3.625478e+09,...,3.625478e+09,0.0500,3.625478e+09,0.0500,9999,"Nordic Functionalism, mid 1900s, Sweden",0.285956,0.447914,0.250,0.90
3729090260,Sweden,Buildings,"Million Program, mid 1900s",3625478156,0.0100,3625478145,0.0500,3.625478e+09,0.0700,3.625478e+09,...,3.625478e+09,0.0100,3.625478e+09,0.0100,9999,"Million Program, mid 1900s, Sweden",0.297697,0.330465,0.180,0.92
3729005194,Sweden,Buildings,"Mixed-City, early 200s",3625478156,0.2200,3625478168,0.0400,3.625478e+09,0.2600,3.625478e+09,...,3.625478e+09,0.2200,3.625478e+09,0.2200,9999,"Mixed-City, early 200s, Sweden",0.079402,0.192080,0.180,0.92
3730430893,Sweden,Buildings,"Detached houses, Wood",3625478178,0.0540,3625478155,0.0250,3.625478e+09,0.0150,3.625478e+09,...,3.625478e+09,0.0050,3.625478e+09,0.0050,9999,"Detached houses, Wood, Sweden",1.097377,0.804598,0.200,0.90
3730430894,UK,Buildings,Central London [sample run],3630433784,0.0540,3630433785,0.1500,3.630434e+09,0.0400,3.630434e+09,...,3.630434e+09,0.0497,3.630434e+09,0.0497,9057,"Central London [sample run], UK",0.464196,0.314830,0.500,0.95
3730430895,UK,Paved,Paved [sample run],3630433788,0.0400,3630433789,0.3000,3.630434e+09,1.2000,3.630434e+09,...,NaN,NaN,NaN,NaN,9057,"Paved [sample run], UK",NaN,NaN,NaN,NaN
3730430892,Sweden,Buildings,Industry,3630433783,0.0100,3625478145,0.0500,3.625478e+09,0.0300,3.625478e+09,...,3.630434e+09,0.0400,3.630434e+09,0.0400,9999,"Industry, Sweden",0.327795,0.659896,0.060,0.95
3730430896,UK,Buildings,"Residential, pre-1900-1944, pitched roof, type...",3625478178,0.0120,3625478158,0.0200,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"9069, 9070","Residential, pre-1900-1944, pitched roof, type...",3.241267,7.179487,0.200,0.90


In [21]:
db[col]['nameOrigin'] = db[col]['Name'].astype(str) + ', ' + db[col]['Origin'].astype(str)
# Filter rows where Surface is 'Buildings'
buildings = db['Spartacus Surface'][db['Spartacus Surface']['Surface'] == 'Buildings']

# Calculate resistances and U-values
for prefix in ['w', 'r']:
    materials = buildings[[f'{prefix}{i}Material' for i in range(1, 6)]].values
    thicknesses = buildings[[f'{prefix}{i}Thickness' for i in range(1, 6)]].values

    try:
        thermal_conductivities = np.vectorize(lambda x: db['Spartacus Material'].loc[x, 'Thermal Conductivity'])(materials)

        resistances = thicknesses / thermal_conductivities
        resistance_bulk = resistances.sum(axis=1)

        u_values = 1 / resistance_bulk
        db['Spartacus Surface'].loc[buildings.index, f'u_value_{prefix}all'] = u_values
    except:
        print(col,',' ,prefix)
        # print(materials)
        # print(lambda x: db['Spartacus Material'].loc[x, 'Thermal Conductivity'])
        print(a)
        a = a+1
        
# Calculate albedo and emissivity
for prop in ['Albedo', 'Emissivity']:
    for prefix in ['w', 'r']:
        if prefix == 'w':
            pr = 'wall'
        elif prefix == 'r':
            pr == 'roof'
        material_col = f'{prefix}1Material'
        db['Spartacus Surface'].loc[buildings.index,f'{prop.lower()}_{pr}'] = db['Spartacus Material'].loc[buildings[material_col], prop].values

IndentationError: unexpected indent (1347434393.py, line 3)

In [6]:
db[col]['nameOrigin'] = db[col]['Name'].astype(str) + ', ' + db[col]['Origin'].astype(str)
    # Filter rows where Surface is 'Buildings'
buildings = db['Spartacus Surface'][db['Spartacus Surface']['Surface'] == 'Buildings']

# Calculate resistances and U-values
for prefix in ['w', 'r']:
    materials = buildings[[f'{prefix}{i}Material' for i in range(1, 6)]].values
    thicknesses = buildings[[f'{prefix}{i}Thickness' for i in range(1, 6)]].values

    thicknesses[np.isnan(thicknesses)] = 0

    for i in range(0,5):
        materials[np.isnan(materials)] = materials[np.nonzero(np.isnan(materials))[0], np.nonzero(np.isnan(materials))[1]-1]

    thermal_conductivities = np.vectorize(lambda x: db['Spartacus Material'].loc[x, 'Thermal Conductivity'])(materials)

    resistances = thicknesses / thermal_conductivities
    resistance_bulk = resistances.sum(axis=1)

    u_values = 1 / resistance_bulk
    db['Spartacus Surface'].loc[buildings.index, f'u_value_{prefix}all'] = u_values

# Calculate albedo and emissivity
for prop in ['Albedo', 'Emissivity']:
    for prefix in ['w', 'r']:
        if prefix == 'w':
            pr = 'wall'
        elif prefix == 'r':
            pr == 'roof'
        material_col = f'{prefix}1Material'
        db['Spartacus Surface'].loc[buildings.index, f'{prop.lower()}_{pr}'] = db['Spartacus Material'].loc[buildings[material_col], prop].values

db['Spartacus Surface']

,Origin,Surface,Name,r1Material,r1Thickness,r2Material,r2Thickness,r3Material,r3Thickness,r4Material,...,w4Material,w4Thickness,w5Material,w5Thickness,Ref,nameOrigin,u_value_wall,u_value_rall,albedo_wall,emissivity_wall
ID,,,,,,,,,,,,,,,,,,,,,
3729073437,Sweden,Buildings,"Traditional Neighbourhood, early 1900s",3625478178,0.0540,3625478155,0.0250,3.625478e+09,0.0250,3.625478e+09,...,3.625478e+09,0.0500,3.625478e+09,0.0500,9999,"Traditional Neighbourhood, early 1900s, Sweden",0.298716,0.433437,0.200,0.90
3729089758,Sweden,Buildings,"Nordic Functionalism, mid 1900s",3625478180,0.0540,3625478155,0.0250,3.625478e+09,0.0250,3.625478e+09,...,3.625478e+09,0.0500,3.625478e+09,0.0500,9999,"Nordic Functionalism, mid 1900s, Sweden",0.285956,0.447914,0.250,0.90
3729090260,Sweden,Buildings,"Million Program, mid 1900s",3625478156,0.0100,3625478145,0.0500,3.625478e+09,0.0700,3.625478e+09,...,3.625478e+09,0.0100,3.625478e+09,0.0100,9999,"Million Program, mid 1900s, Sweden",0.297697,0.330465,0.180,0.92
3729005194,Sweden,Buildings,"Mixed-City, early 200s",3625478156,0.2200,3625478168,0.0400,3.625478e+09,0.2600,3.625478e+09,...,3.625478e+09,0.2200,3.625478e+09,0.2200,9999,"Mixed-City, early 200s, Sweden",0.079402,0.192080,0.180,0.92
3730430893,Sweden,Buildings,"Detached houses, Wood",3625478178,0.0540,3625478155,0.0250,3.625478e+09,0.0150,3.625478e+09,...,3.625478e+09,0.0050,3.625478e+09,0.0050,9999,"Detached houses, Wood, Sweden",1.097377,0.804598,0.200,0.90
3730430894,UK,Buildings,Central London [sample run],3630433784,0.0540,3630433785,0.1500,3.630434e+09,0.0400,3.630434e+09,...,3.630434e+09,0.0497,3.630434e+09,0.0497,9057,"Central London [sample run], UK",0.464196,0.314830,0.500,0.95
3730430895,UK,Paved,Paved [sample run],3630433788,0.0400,3630433789,0.3000,3.630434e+09,1.2000,3.630434e+09,...,-9.990000e+02,-999.0000,-9.990000e+02,-999.0000,9057,"Paved [sample run], UK",NaN,NaN,NaN,NaN
3730430892,Sweden,Buildings,Industry,3630433783,0.0100,3625478145,0.0500,3.625478e+09,0.0300,3.625478e+09,...,3.630434e+09,0.0400,3.630434e+09,0.0400,9999,"Industry, Sweden",0.327795,0.659896,0.060,0.95
3730430896,UK,Buildings,"Residential, pre-1900-1944, pitched roof, type...",3625478178,0.0120,3625478158,0.0200,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"9069, 9070","Residential, pre-1900-1944, pitched roof, type...",3.241267,7.179487,0.200,0.90


In [7]:
buildings = db['Spartacus Surface'][db['Spartacus Surface']['Surface'] == 'Buildings']
buildings
for prefix in ['w', 'r']:
    materials = buildings[[f'{prefix}{i}Material' for i in range(1, 4)]].values
    thicknesses = buildings[[f'{prefix}{i}Thickness' for i in range(1, 4)]].values

thicknesses


array([[0.054 , 0.025 , 0.025 ],
       [0.054 , 0.025 , 0.025 ],
       [0.01  , 0.05  , 0.07  ],
       [0.22  , 0.04  , 0.26  ],
       [0.054 , 0.025 , 0.015 ],
       [0.054 , 0.15  , 0.04  ],
       [0.01  , 0.05  , 0.03  ],
       [0.012 , 0.02  ,    nan],
       [0.012 , 0.02  ,    nan],
       [0.0012, 0.02  , 0.0125],
       [0.012 , 0.0123, 0.0125],
       [0.012 , 0.09  , 0.0125],
       [0.0012, 0.019 , 0.009 ],
       [0.012 , 0.02  , 0.06  ],
       [0.012 , 0.02  , 0.113 ],
       [0.012 , 0.247 , 0.0125],
       [0.0012, 0.019 , 0.009 ],
       [0.0012, 0.089 , 0.075 ],
       [0.0012, 0.089 , 0.075 ],
       [0.0012, 0.02  , 0.0125]])

In [ ]:
                thermal_conductivities = np.vectorize(lambda x: db['Spartacus Material'].loc[x, 'Thermal Conductivity'])(materials)
            
                resistances = thicknesses / thermal_conductivities
                resistance_bulk = resistances.sum(axis=1)

                u_values = 1 / resistance_bulk
                db['Spartacus Surface'].loc[buildings.index, f'u_value_{prefix}all'] = u_values
            except:
                print(col,',' ,prefix)
        # Calculate albedo and emissivity
        for prop in ['Albedo', 'Emissivity']:
            for prefix in ['w', 'r']:
                material_col = f'{prefix}1Material'
                db['Spartacus Surface'].loc[buildings.index, f'{prop.lower()}_{prefix}all'] = db['Spartacus Material'].loc[buildings[material_col], prop].values
#     else:
#         print(col)
#         db[col]['nameOrigin'] = db[col]['Name'].astype(str) + ', ' + db[col]['Origin'].astype(str)



In [12]:
db['Spartacus Surface']

,Origin,Surface,Name,r1Material,r1Thickness,r2Material,r2Thickness,r3Material,r3Thickness,r4Material,...,w5Material,w5Thickness,Ref,nameOrigin,albedo_wall,albedo_rall,emissivity_wall,emissivity_rall,u_value_wall,u_value_rall
ID,,,,,,,,,,,,,,,,,,,,,
3729073437,Sweden,Buildings,"Traditional Neighbourhood, early 1900s",3625478178,0.0540,3625478155,0.0250,3.625478e+09,0.0250,3.625478e+09,...,3.625478e+09,0.0500,9999,"Traditional Neighbourhood, early 1900s, Sweden",0.22,0.200,0.90,0.90,0.298716,0.433437
3729089758,Sweden,Buildings,"Nordic Functionalism, mid 1900s",3625478180,0.0540,3625478155,0.0250,3.625478e+09,0.0250,3.625478e+09,...,3.625478e+09,0.0500,9999,"Nordic Functionalism, mid 1900s, Sweden",0.32,0.250,0.95,0.90,0.285956,0.447914
3729090260,Sweden,Buildings,"Million Program, mid 1900s",3625478156,0.0100,3625478145,0.0500,3.625478e+09,0.0700,3.625478e+09,...,3.625478e+09,0.0100,9999,"Million Program, mid 1900s, Sweden",0.20,0.180,0.90,0.92,0.297697,0.330465
3729005194,Sweden,Buildings,"Mixed-City, early 200s",3625478156,0.2200,3625478168,0.0400,3.625478e+09,0.2600,3.625478e+09,...,3.625478e+09,0.2200,9999,"Mixed-City, early 200s, Sweden",0.30,0.180,0.92,0.92,0.079402,0.192080
3730430893,Sweden,Buildings,"Detached houses, Wood",3625478178,0.0540,3625478155,0.0250,3.625478e+09,0.0150,3.625478e+09,...,3.625478e+09,0.0050,9999,"Detached houses, Wood, Sweden",0.30,0.200,0.95,0.90,1.097377,0.804598
3730430894,UK,Buildings,Central London [sample run],3630433784,0.0540,3630433785,0.1500,3.630434e+09,0.0400,3.630434e+09,...,3.630434e+09,0.0497,9057,"Central London [sample run], UK",0.50,0.500,0.95,0.95,0.464196,0.314830
3730430895,UK,Paved,Paved [sample run],3630433788,0.0400,3630433789,0.3000,3.630434e+09,1.2000,3.630434e+09,...,NaN,NaN,9057,"Paved [sample run], UK",NaN,NaN,NaN,NaN,NaN,NaN
3730430892,Sweden,Buildings,Industry,3630433783,0.0100,3625478145,0.0500,3.625478e+09,0.0300,3.625478e+09,...,3.630434e+09,0.0400,9999,"Industry, Sweden",0.37,0.060,0.92,0.95,0.327795,0.659896
3730430896,UK,Buildings,"Residential, pre-1900-1944, pitched roof, type...",3625478178,0.0120,3625478158,0.0200,NaN,NaN,NaN,...,NaN,NaN,"9069, 9070","Residential, pre-1900-1944, pitched roof, type...",0.43,0.200,0.90,0.90,3.241267,7.179487


In [13]:
db['NonVeg']

,Surface,Origin,Name,Albedo,Emissivity,Water Storage,Water State,Drainage,Spartacus Surface,OHMSummerWet,OHMSummerDry,OHMWinterWet,OHMWinterDry,ESTM,ANOHM
ID,,,,,,,,,,,,,,,
20240002,Buildings,Helsinki,Kumpula,412,402,435,430,460,3.730431e+09,502,502,502,502,529.0,512.0
20240005,Buildings,Sweden,"Brick House, Traditional Neighbourhood",412,405,432,424,455,3.729073e+09,502,505,502,502,522.0,512.0
20240006,Buildings,Sweden,"Nordic Functionalism, Wood",4126,402,432,424,452,3.729090e+09,502,502,502,502,522.0,512.0
20240007,Buildings,Sweden,Concrete (Million Program),412,402,435,424,455,3.729090e+09,502,502,502,505,5210.0,512.0
20240008,Buildings,Sweden,"Detached houses, Wood",4126,402,435,424,455,3.730431e+09,502,502,502,502,522.0,512.0
20240009,Buildings,Sweden,Mixed-City,412,402,435,424,455,3.729005e+09,502,502,502,502,522.0,512.0
20240010,Paved,Sweden,"Asphalt, mixed city",4119,4011248,434,424,451,3.730431e+09,501,501,501,501,5212.0,514.0
20240012,Buildings,Sweden,Industry,4125,402,432,421,452,3.730431e+09,5024,5024,5024,5024,529.0,515.0
20240013,Buildings,Beijing,City centre,4112,405,435,421,460,NaN,5045,5045,5045,5045,NaN,NaN


In [11]:
dataframes = db.copy()

# Funktion för att minska antalet siffror i db-code

def remove_third_and_fourth_digits(df, suffixes):
    df = df.reset_index()  # Återställ indexet till en kolumn
    df.replace('', np.nan, inplace=True)  # Fyll tomma celler med NaN
    for col in df.columns:
        if any(col.endswith(suffix) for suffix in suffixes):
            df[col] = df[col].apply(lambda x: float(str(x)[:2] + str(x)[4:]) if pd.notnull(x) and str(x).replace('.', '', 1).isdigit() and len(str(x)) == 10 else x)
    df = df.set_index(df.columns[0])  # Sätt tillbaka den första kolumnen som index
    return df

# Lista över suffix som ska kontrolleras
suffixes_to_check = ['ID','Paved','Buildings','Bare Soil','Grass','Evergreen Tree','Deciduous Tree','Water','Biogen','Conductance',
                     'Albedo','Emissivity','Water Storage','Leaf Area Index','Water State','Leaf Growth Power','Drainage',
                     'Max Vegetation Conductance','Porosity','OHMSummerWet','OHMSummerDry','OHMWinterWet','OHMWinterDry','ANOHM','ESTM',
                     'Spartacus Surface', 'Material','Ref'

                    ]

# Iterera genom varje dataframe och minska antalet siffror i de angivna kolumnerna
for sheet_name, df in dataframes.items():
    try:
        dataframes[sheet_name] = remove_third_and_fourth_digits(df, suffixes_to_check)
        
    except:
        print(sheet_name)
        
# Funktion för att kontrollera dubbletter i index
def check_duplicate_indices(dataframes):
    duplicates = {}
    for sheet_name, df in dataframes.items():
        duplicate_indices = df.index[df.index.duplicated()].unique()
        if len(duplicate_indices) > 0:
            duplicates[sheet_name] = duplicate_indices
    return duplicates

# Kontrollera dubbletter i index för varje dataframe
duplicates = check_duplicate_indices(dataframes)

# Visa resultatet
if duplicates:
    for sheet_name, duplicate_indices in duplicates.items():
        print(f"Sheet: {sheet_name} har dubbletter i index: {duplicate_indices}")
else:
    print("Inga dubbletter hittades i index.")

save_to_db('C:/temp/newdb.xlsx', dataframes)




Profiles
Inga dubbletter hittades i index.


,Surface,Origin,Name,Albedo,Emissivity,Water Storage,Leaf Area Index,Water State,Leaf Growth Power,Drainage,Max Vegetation Conductance,Porosity,OHMSummerWet,OHMSummerDry,OHMWinterWet,OHMWinterDry,ANOHM,ESTM,BiogenCO2,nameOrigin
ID,,,,,,,,,,,,,,,,,,,,
24000001.0,Evergreen Tree,Tropics,Broad leaf,417,407,437,317,426,321,457,333,3410.0,5010,5010,507,507,NaN,NaN,NaN,"Broad leaf, Tropics"
24000002.0,Grass,Vancouver,Irrigated [sample run],419,4011258,439,319,428,326,459,331,3411.0,509,509,509,509,NaN,NaN,NaN,"Irrigated [sample run], Vancouver"
24000003.0,Evergreen Tree,"Helsinki, Finland","Mixed Evergreen Trees, Järvi et al (2014)",417,407,437,317,426,321,457,333,3410.0,507,508,508,508,NaN,NaN,NaN,"Mixed Evergreen Trees, Järvi et al (2014), Hel..."
24000004.0,Deciduous Tree,"Helsinki, Finland","Mixed Dec, Järvi et al (2014)",4127,408,438,318,427,322,458,334,342.0,508,508,508,508,NaN,NaN,NaN,"Mixed Dec, Järvi et al (2014), Helsinki, Finland"
24000005.0,Grass,"Helsinki, Finland","Irrigated Grass, Järvi et al (2014)",4129,4011258,439,3117,428,323,4515,331,3411.0,508,508,508,508,NaN,NaN,NaN,"Irrigated Grass, Järvi et al (2014), Helsinki,..."
24000006.0,Grass,"Helsinki, Finland","Unirrigated Grass, Järvi et al (2014)",4129,4011258,439,3117,428,323,4512,332,3411.0,508,508,508,508,NaN,NaN,NaN,"Unirrigated Grass, Järvi et al (2014), Helsink..."
24000007.0,Grass,"Swindon, UK","Grass, Ward et al (2013)",419,4011258,439,3117,428,323,459,332,3411.0,508,508,508,508,NaN,NaN,NaN,"Grass, Ward et al (2013), Swindon, UK"
24000008.0,Deciduous Tree,"Swindon, UK","Mixed Dec, Ward (2013)",418,408,438,318,427,322,4511,334,342.0,508,508,508,508,NaN,NaN,NaN,"Mixed Dec, Ward (2013), Swindon, UK"
24000009.0,Evergreen Tree,"Beijing, China",Mixed Evergreen Trees,417,407,437,3123,421,327,461,336,3410.0,507,507,507,507,NaN,NaN,NaN,"Mixed Evergreen Trees, Beijing, China"


In [21]:
import pandas as pd

# Load the Excel file
file_path = r'C:\GitHub\suews_database_manager\data\database_b.xlsx'  # Replace with your actual file path
xls = pd.ExcelFile(file_path)

def standardize_code(code):
    # Check for NaN, None, or empty
    if pd.isna(code) or code is None or len(str(code)) == 0:
        return code  # Do nothing for NaN, None, or empty
    
    # Attempt to convert to integer, ignoring decimals
    try:
        code_int = int(float(code))  # Convert to float first to handle any decimal points
    except (ValueError, TypeError):
        return code  # If conversion fails, return the original code

    code_str = str(code_int)  # Convert back to string for processing

    if len(code_str) == 8:
        return code_int  # Return as integer
    elif len(code_str) == 3:
        return int(code_str[:2] + '24' + '000' + code_str[2])
    elif len(code_str) == 4:
        return int(code_str[:2] + '24' + '00' + code_str[2:])
    elif len(code_str) == 5:
        return int(code_str[:2] + '24' + '0' + code_str[2:])
    elif len(code_str) == 6:
        return int(code_str[:2] + '24' + code_str[2:])
    elif len(code_str) == 7:
        return int(code_str[:2] + '24' + code_str[3:])
    elif len(code_str) > 8:
        return int(code_str[:2] + '24' + code_str[-4:])
    else:
        return int(code_str[:8])  # Ensure it's an integer





# List of columns that include codes
code_columns = [
    "ID", "SnowClearingProfWD", "SnowClearingProfWE", "AnthropogenicCode", "IrrigationCode",
    "WaterUseProfManuWD", "WaterUseProfManuWE", "WaterUseProfAutoWD", "WaterUseProfAutoWE",
    "Paved", "Buildings", "Bare Soil", "Grass", "Evergreen Tree", "Deciduous Tree",
    "SoilTypeCode", "Water", "SnowCode", "Vegetation Growth", "Conductance", "Albedo",
    "Emissivity", "Water Storage", "Drainage", "Spartacus Surface", "OHMSummerWet",
    "OHMSummerDry", "OHMWinterWet", "OHMWinterDry", "SnowLimPatch", "Leaf Area Index",
    "Leaf Growth Power", "Drainage", "Max Vegetation Conductance", "Porosity",
    "Water Storage", "Water State", "Ref","r1Material","r2Material","r3Material","r4Material","r5Material",
    "w1Material","w2Material","w3Material","w4Material","w5Material", "Biogen CO2",
    "TrafficRate_WD","TrafficRate_WE","EnergyUseProfWD","EnergyUseProfWE","ActivityProfWD","ActivityProfWE","PopProfWD","PopProfWE"

]


new_file_path = r'C:\GitHub\suews_database_manager\data\database_d.xlsx'  # Replace with your actual file path


# Create a dictionary to store DataFrames
df_dict = {}

# Loop through each sheet and apply standardization
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    
    # Apply standardization to the specified columns
    for col in code_columns:
        if col in df.columns:
            df[col] = df[col].apply(standardize_code)
    
    # Store the modified DataFrame in the dictionary
    df_dict[sheet_name] = df

# Save the DataFrame dictionary to a new Excel file
with pd.ExcelWriter(new_file_path, engine='openpyxl') as writer:
    for sheet_name, df in df_dict.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Consistency applied to all specified columns in all sheets. The results are saved to {new_file_path}.")
# writer = pd.ExcelWriter(new_file_path, engine='openpyxl')

Consistency applied to all specified columns in all sheets. The results are saved to C:\GitHub\suews_database_manager\data\database_d.xlsx.


In [16]:
code_str = str(401124)

In [18]:
a = int(code_str[:2] + '24' + code_str[2:])

len(str(a))

8

In [20]:
code_str

'401124'